In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [2]:
spark = SparkSession.builder \
    .appName("dm_customer") \
    .config("spark.cores.max", "1") \
    .config("spark.executor.memory", "2g") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.iceberg.type", "hive") \
    .config("spark.sql.catalog.iceberg.uri", "thrift://hive-metastore:9083") \
    .config("spark.sql.catalog.iceberg.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \
    .config("spark.sql.catalog.iceberg.warehouse", "s3a://warehouse/") \
    .config("spark.sql.catalog.iceberg.s3.endpoint", "http://minio:9000") \
    .getOrCreate()

spark.sparkContext.setLogLevel('ERROR')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/23 18:00:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read \
    .format('iceberg') \
    .load('iceberg.bronze.reviews')
df.printSchema()
# df.show()

root
 |-- product_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- content: string (nullable = true)
 |-- status: string (nullable = true)
 |-- thank_count: string (nullable = true)
 |-- score: string (nullable = true)
 |-- new_score: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_region: string (nullable = true)
 |-- customer_created_time: string (nullable = true)
 |-- customer_purchased: string (nullable = true)
 |-- customer_purchased_at: string (nullable = true)
 |-- customer_total_review: string (nullable = true)
 |-- customer_total_thank: string (nullable = true)
 |-- attributes: string (nullable = true)
 |-- spid: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- seller_name: string (nullable = 

In [4]:
df = df.select(
    'customer_id',
    'customer_name',
    'customer_region',
    'customer_created_time',
    'customer_total_review',
    'customer_total_thank',
    'ngay_cap_nhat'
)
df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_region: string (nullable = true)
 |-- customer_created_time: string (nullable = true)
 |-- customer_total_review: string (nullable = true)
 |-- customer_total_thank: string (nullable = true)
 |-- ngay_cap_nhat: timestamp (nullable = true)



In [5]:
w = (
    Window
    .partitionBy("customer_id")
    .orderBy(F.col("ngay_cap_nhat").desc())
)

df = (
    df
    .withColumn("rn", F.row_number().over(w))
    .filter(F.col("rn") == 1)
    .drop("rn", "ngay_cap_nhat")
)

In [6]:
df = df.filter(col("customer_id").isNotNull())

In [7]:
df = df.withColumn("ngay_cap_nhat", current_timestamp())

In [8]:
df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_region: string (nullable = true)
 |-- customer_created_time: string (nullable = true)
 |-- customer_total_review: string (nullable = true)
 |-- customer_total_thank: string (nullable = true)
 |-- ngay_cap_nhat: timestamp (nullable = false)



In [9]:
df.write \
    .format('iceberg') \
    .mode('overwrite') \
    .saveAsTable('iceberg.silver.dm_customer')

In [10]:
spark.stop()